# Попробуем реализовать проверку скорости печати в терминале

## Что для этого нужно?

In [ ]:
import curses
import time 
from pygame import mixer
from faker import Faker

## Генерация текста для проверки

In [ ]:
# !pip install faker
# !pip install pygame

In [ ]:
from faker import Faker
fake = Faker(locale='en')

In [ ]:
fake.paragraph(nb_sentences=5)

## Захват экрана (не запускай код в юпитере, убьет)

initscr() возвращает оконный объект, который является экраном терминала.
Когда происходит инициализация, ширина и высота терминала возвращается с помощью initscr. Она равна физическим высоте и ширине

In [ ]:
import curses

stdscr = curses.initscr()

refresh() - функция для обновления дисплея

wrapper() - функция, которая принимает callable объект и выполняет ее.

addstr() - функция для добавления текста на экран (принимает позицию текста на экране и текст)

In [ ]:
import curses 

def main(stdscr):
    stdscr = curses.initscr()
    text = "Hello!"
    
    stdscr.addstr(0, 5, text)
    stdscr.refresh()
    
    curses.napms(1000)
    
curses.wrapper(main)

## Получение текста с клавиатуры

In [ ]:
keyPress = stdscr.getkey()

## Отображение текста

curses.start_color() - нужно для отображения текста нужного цвета

init_pair(pair_num, foreground_color, background_color) нужно для инициализации пар цветов (цвет текста и фона)

In [ ]:
curses.start_color()
curses.init_pair(1, curses.COLOR_MAGENTA, curses.COLOR_BLACK)

## Отображение информации о прошедшем времени и скорости печати

In [ ]:
stdscr.addstr(text)

stdscr.addstr(5, 0, f"Typing Speed in words per minute (wpm): {wpm}")
stdscr.addstr(4, 0, f"Typing Speed in characters per minute : {cpm}")
stdscr.addstr(6, 0, f"Seconds elapsed : {round(seconds_passed, 2)} seconds") 
stdscr.addstr(7, 0, f"Errors count : {errors}") 

## Отображение в нужных цветах введенного текста

In [ ]:
for index, letter in enumerate(typed_text):
    correct_char = text[index]
    if letter != correct_char:
        text_color = curses.color_pair(2)
    else:
        text_color = curses.color_pair(1)

    stdscr.addstr(index // 123, index % 123, letter, text_color)

## Подсчет характеристик

In [ ]:
seconds_passed = max((time.time() - start_time), 1)
cpm = int(len(typed_text) / (seconds_passed / 60))
wpm = round((cpm/5), 2)

## Бонусом добавление фонового аудио

In [1]:
from pygame import mixer

pygame 2.5.2 (SDL 2.28.3, Python 3.11.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
 mixer.init() 

In [6]:
if not mixer.music.get_busy():
    mixer.music.load('src/shit_music.mp3')
    mixer.music.play()

## Добавление аудио для ошибок

In [20]:
walk_channel = mixer.Channel(2)
walk_sound = mixer.Sound("src/ouch.wav")
walk_channel.play(walk_sound)

## В результате будет что-то такое

In [ ]:
import curses
import time 
from pygame import mixer
from faker import Faker

In [ ]:
fake = Faker(locale='en')
text = fake.paragraph(nb_sentences=5)
errors = 0


In [ ]:
def show_speed(stdscr, text, typed_text, cpm, wpm, seconds_passed):
    global errors
    
    stdscr.addstr(text)
    stdscr.addstr(5, 0, f"Typing Speed in words per minute (wpm): {wpm}")
    stdscr.addstr(4, 0, f"Typing Speed in characters per minute : {cpm}")
    stdscr.addstr(6, 0, f"Seconds elapsed : {round(seconds_passed, 2)} seconds") 
    stdscr.addstr(7, 0, f"Errors count : {errors}") 


    for index, letter in enumerate(typed_text):
        correct_char = text[index]
        if letter != correct_char:
            text_color = curses.color_pair(2)
        else:
            text_color = curses.color_pair(1)

        stdscr.addstr(index // 123, index % 123, letter, text_color)
    

In [ ]:
def speed(stdscr):
    global text
    global errors

    
    mixer.init() 
    
    
    typed_text = []
    wpm = 0
    start_time = time.time()
    errors = 0
    curses.flushinp()

    while True:
        if not mixer.music.get_busy():
            mixer.music.load('src/shit_music.mp3')
            mixer.music.play()
        seconds_passed = max((time.time() - start_time), 1)
        cpm = int(len(typed_text) / (seconds_passed/60))
        wpm = round((cpm/5), 2)

        stdscr.clear()
        show_speed(stdscr, text, typed_text, cpm, wpm, seconds_passed)
        stdscr.refresh()

        if "".join(typed_text) == text:
            stdscr.nodelay(False)
            break
        
        try:
            keyPress = stdscr.getkey()
        except:
            continue


        if len(typed_text) == len(text):
            break

        if ord(keyPress) == 27:
            break 
        
        if keyPress in ("KEY_BACKSPACE", '\b', '\x7f'):
            if len(typed_text) > 0:
                typed_text.pop()

        elif len(typed_text) < len(text):
            if keyPress != text[len(typed_text)]:
                walk_channel = mixer.Channel(2)
                walk_sound = mixer.Sound("src/ouch.wav")
                walk_channel.play(walk_sound)
                errors += 1
            typed_text.append(keyPress)

In [ ]:
def main(stdscr):
    global text
    curses.init_pair(1, curses.COLOR_MAGENTA, curses.COLOR_BLACK)
    curses.init_pair(2, curses.COLOR_CYAN, curses.COLOR_BLACK)
    curses.init_pair(3, curses.COLOR_WHITE, curses.COLOR_BLACK)
    curses.init_pair(4, curses.COLOR_RED, curses.COLOR_BLACK)
    
    while True:
        speed(stdscr)

        #color blinking
        for i in range(0,2):
            stdscr.addstr(0, 0, text, curses.color_pair(2))
            stdscr.refresh()
            curses.napms(400)
            stdscr.addstr(0, 0, text, curses.color_pair(4))
            stdscr.refresh()
            curses.napms(400)
            stdscr.addstr(0, 0, text, curses.color_pair(1))
            stdscr.refresh()
            curses.napms(400)


        stdscr.addstr(8, 0, "Yay!! Press any key to continue. \nTo exit press the escape button.")

        curses.flushinp()
        keyPress = stdscr.getkey()
        
        #break if esc key is pressed
        if ord(keyPress) == 27:
            break

        text = fake.paragraph(nb_sentences=5)


In [ ]:
curses.wrapper(main)